In [ ]:
import pandas as pd
import re
import requests
import os

from dotenv import load_dotenv

#.env 파일 로드
load_dotenv()
kakao_api_key = os.getenv("kakao_api_key")


### 위도, 경도, 행정구역 컬럼 추가

In [6]:
def pre_col_gen(df):
    # 중복된 '공고번호' 제거 (첫 번째 항목만 남김)
    df_copy = df.reset_index(drop=True).drop_duplicates(subset="공고번호", keep='first', ignore_index=True)
    
    # '정제_공급위치' 열 생성 후 간단한 전처리
    df_copy["정제_공급위치"] = df_copy["공급위치"].apply(lambda address: re.sub(r'\([^)]*\)', '', address).replace('블록', '').replace('일원', '').replace('공동', '').replace('일반', '').replace('  ', ' ').strip())
    
    # 필요한 열만 선택
    df_copy = df_copy[["공고번호", "정제_공급위치"]]
    
    return df_copy

def process_address_data(df):
    # 카카오맵 API 사용해서 위도 경도 추출
    kakao_api_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}

    def get_lat_lon_kakao(address):
        try:
            response = requests.get(kakao_api_url, headers=headers, params={"query": address})
            if response.status_code == 200:
                result = response.json()
                if result['documents']:
                    lat = result['documents'][0]['y']
                    lon = result['documents'][0]['x']
                    return lat, lon
            return None, None
        except Exception as e:
            print(f"Error fetching data for address {address}: {e}")
            return None, None

    df['위도'], df['경도'] = zip(*df['정제_공급위치'].apply(lambda x: get_lat_lon_kakao(x)))

    return df

def process_address(df):
    
    def split_and_update_address(address):
        parts = address.split(' ')
        
        if len(parts) >= 3:
            sido = parts[0]
            sigungu = parts[1]
            eupmyeondong = parts[2]
            updated_address = f"{sido} {sigungu} {eupmyeondong}"
            return sido, sigungu, eupmyeondong, updated_address
        else:
            return None, None, None, address
    
    # 주소 분리 및 업데이트
    df[['시도', '시군구', '읍면동', '정제_공급위치_업데이트']] = df['정제_공급위치'].apply(lambda x: pd.Series(split_and_update_address(x)))
    
    # 기존 정제_공급위치 열을 업데이트함
    df['정제_공급위치'] = df['정제_공급위치_업데이트']
    
    # 불필요한 열 제거
    df = df.drop(columns=['정제_공급위치_업데이트'])
    
    return df


def lat_lon_pipeline(df):
    df_copy = pre_col_gen(df)

    # Kakao API로 위도와 경도 가져오기
    df_copy = process_address_data(df_copy)
    
    # 주소 분리 및 업데이트 
    df_copy = process_address(df_copy)

    # 위도와 경도가 None인 행만 선택하여 다시 처리
    df_none = df_copy[(df_copy['위도'].isnull()) & (df_copy['경도'].isnull())].copy()
    
    if not df_none.empty:
        df_none_processed = process_address_data(df_none)
        df_copy.loc[df_none.index, ['위도', '경도']] = df_none_processed[['위도', '경도']]
    
    df_copy.drop("정제_공급위치", axis=1, inplace=True)
    
    return df_copy


In [ ]:
df = pd.read_csv("train-250314-test.csv", encoding='cp949')
df_copy = lat_lon_pipeline(df)
df_merged = pd.merge(df, df_copy, on='공고번호', how='left')

In [11]:
df_merged.to_csv('병합_청약매물_목록_정보 (서울, 경기, 인천).csv', index=False, encoding='cp949')

In [10]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18860 entries, 0 to 18859
Data columns (total 41 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   공고번호          18860 non-null  int64  
 1   주택명           18860 non-null  object 
 2   공급지역코드        18860 non-null  int64  
 3   공급지역명         18860 non-null  object 
 4   공급위치우편번호      18860 non-null  int64  
 5   공급위치          18860 non-null  object 
 6   공급규모          18860 non-null  int64  
 7   청약접수종료일       18860 non-null  object 
 8   투기과열지구        18860 non-null  object 
 9   조정대상지역        18860 non-null  object 
 10  분양가상한제        18860 non-null  object 
 11  정비사업          18860 non-null  object 
 12  공공주택지구        18860 non-null  object 
 13  대규모택지개발지구     18860 non-null  object 
 14  수도권내민영공공주택지구  18860 non-null  object 
 15  신청자 30대 이하    18860 non-null  int64  
 16  신청자 40대       18860 non-null  int64  
 17  신청자 50대       18860 non-null  int64  
 18  신청자 60대 이상    18860 non-nu